In [1]:
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re

import os
from os.path import exists, join, isfile, abspath
import time
import csv
import json
import datetime
from pathlib import Path
import shutil
import PyPDF2
import pandas as pd

In [2]:
download_path = abspath("./export")
cata_path = abspath("./Catalogues")
history_record_file = "./history_record.json"
weekly_record_file = "./weekly_catalogues_tracking.csv"
history_record = {}
weekly_record = []
tmp_print_pdf = "tmp_catalogue.pdf"

try:
    with open(history_record_file, "r", encoding='utf-8') as recordfile:
        history_record = json.load(recordfile)
except:
    print("No history record")

No history record


In [3]:
today = datetime.date.today()
monday = today + datetime.timedelta( (7-today.weekday()) % 7 )
last_day_check = today == monday
monday_str = monday.strftime("%y%m%d")
monday_ref_str = monday.strftime("%Y-%m-%d")

In [4]:
week_cata_path = join(cata_path, monday_str)
Path(week_cata_path).mkdir(parents=True, exist_ok=True)

In [5]:
ff_options=Options()
ff_options.add_argument("--disable-infobars")
ff_options.add_argument("--disable-extensions")
ff_options.add_argument("--disable-popup-blocking")

ff_options.set_preference("browser.download.dir", download_path)
ff_options.set_preference("browser.download.folderList",2)
ff_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/plain,application/octet-stream,application/pdf,application/x-pdf,application/vnd.pdf")
ff_options.set_preference("browser.download.manager.showWhenStarting", False)
ff_options.set_preference("browser.helperApps.neverAsk.openFile","text/plain,application/octet-stream,application/pdf,application/x-pdf,application/vnd.pdf")
ff_options.set_preference("browser.helperApps.alwaysAsk.force", False)
ff_options.set_preference("browser.download.manager.useWindow", False)
ff_options.set_preference("browser.download.manager.focusWhenStarting", False)
ff_options.set_preference("browser.helperApps.neverAsk.openFile", "")
ff_options.set_preference("browser.download.manager.alertOnEXEOpen", False)
ff_options.set_preference("browser.download.manager.showAlertOnComplete", False)
ff_options.set_preference("browser.download.manager.closeWhenDone", True)

ff_options.set_preference("print_printer", "Microsoft Print to PDF")
ff_options.set_preference("print.always_print_silent", True)
ff_options.set_preference("print.show_print_progress", False)
ff_options.set_preference('print.save_as_pdf.links.enabled', True)
ff_options.set_preference("print.printer_Microsoft_Print_to_PDF.print_to_file", True)
ff_options.set_preference('print.printer_Microsoft_Print_to_PDF.print_to_filename',join(download_path, tmp_print_pdf))

ff_options.set_preference("pdfjs.disabled", True)

driver = Firefox(options=ff_options)

In [ ]:
if not exists(weekly_record_file):
    with open(weekly_record_file, "a", encoding="utf-8", newline="") cata_tracking:
        cata_tracking_writer = csv.writer(cata_tracking)
        cata_tracking_writer.writerow(['Retailer Name','Filename', 'Date Reference']

cata_tracking = open(weekly_record_file, "a", encoding="utf-8", newline="")
cata_tracking_writer = csv.writer(cata_tracking)

cata_tracking_history = pd.read_csv(weekly_record_file, header=0)

In [6]:
# Amcal - https://www.amcal.com.au/online-catalogue

def get_cata_from_amcal(driver, retailer_url, retailer_name, cata_tracking_writer, last_day_check):
    if retailer_name not in history_record.keys():
        history_record[retailer_name] = []

    driver.get(retailer_url)
    wait_time = WebDriverWait(driver, 10)
    try:
        fileurl = wait_time.until(EC.presence_of_element_located((By.XPATH, '//iframe[@id="catalogIframe"]'))).get_attribute("src")
        filename = fileurl.split("/")[-1]
        filepath = join(download_path, filename)
        while not exists(filepath):
            time.sleep(2)

        if isfile(filepath):
            if fileurl not in history_record[retailer_name]:
                history_record[retailer_name].append(fileurl)
                cata_filepath = join(week_cata_path, filename)
                if not exists(cata_filepath):
                    shutil.copy(filepath, cata_filepath)
                    cata_tracking_writer.writerow([retailer_name, filename, monday_ref_str])
            while exists(filepath):
                try:
                    os.remove(filepath)
                except:
                    time.sleep(1)
                    continue
    except TimeoutException:
        print(retailer_name, ": ", "No new catalogue available")
        if len(cata_tracking_history[cata_tracking_history['Retailer Name']==retailer_name, cata_tracking_history['Date Reference']==monday_ref_str]) == 0 and last_day_check:
            cata_tracking_writer.writerow([retailer_name, "No new catalogue available", monday_ref_str])

In [7]:
# Salefinder - https://salefinder.com.au

def get_cata_from_salefinder(driver, retailer_url, retailer_name, cata_tracking_writer, last_day_check):
    if retailer_name not in history_record.keys():
        history_record[retailer_name] = []
    
    driver.get(retailer_url)
    wait_time = WebDriverWait(driver, 10)
    try:
        cata_info = []
        wait_time.until(EC.presence_of_element_located((By.XPATH, '//div[@class="retailer-catalogue"]')))
        cata_list = driver.find_elements(By.XPATH, '//div[@class="retailer-catalogue"]')
        print(retailer_name, ": ", len(cata_list), "catalogue(s)")
        for cata in cata_list:
            cata_href = cata.find_element(By.XPATH, 'a').get_attribute('href')
            cata_name = cata.find_element(By.XPATH, 'div[@class="catalogue-name"]').text
            cata_date = cata.find_element(By.XPATH, 'div[@class="catalogue-date"]').text
            cata_info.append([cata_href, cata_name, cata_date])

        for cata in cata_info:
            cata_href = cata[0]
            cata_name = cata[1]
            cata_date = cata[2]
            
            driver.get(cata_href)
            print_all_href = wait_time.until(EC.presence_of_element_located((By.XPATH, '//a[@id="sf-catalogue-print-all"]'))).get_attribute("href")
            driver.get(print_all_href)
            
            filepath = join(download_path, tmp_print_pdf)
            while not exists(filepath):
                time.sleep(2)
            while True:
                try:
                    with open(filepath, "rb") as pdf_file:
                        PyPDF2.PdfFileReader(pdf_file)
                    break
                except:
                    time.sleep(5)
                    
            if isfile(filepath):
                filename = retailer_name + "_" + cata_name + "_" + cata_date
                filename = re.sub(' +', ' ', filename)
                filename = filename.strip()

                
                if cata_href not in history_record[retailer_name]:
                    history_record[retailer_name].append(cata_href)
                    cata_filepath = join(week_cata_path, filename + ".pdf")
                    if not exists(cata_filepath):
                        shutil.copy(filepath, cata_filepath)
                        cata_tracking_writer.writerow([retailer_name, filename + ".pdf", monday_ref_str])
                    else:
                        index = 1
                        new_filename = filename + " ({0})".format(index)
                        while exists(join(week_cata_path, new_filename + ".pdf")):
                            index += 1
                        

                while exists(filepath):
                    try:
                        os.remove(filepath)
                    except:
                        time.sleep(1)
            
    except TimeoutException:
        print(retailer_name, ": ", "No new catalogue available")
        if len(cata_tracking_history[cata_tracking_history['Retailer Name']==retailer_name, cata_tracking_history['Date Reference']==monday_ref_str]) == 0 and last_day_check:
            cata_tracking_writer.writerow([retailer_name, "No new catalogue available", monday_ref_str])

In [8]:
# -------- Amcal --------

# Amcal
get_cata_from_amcal(driver, "https://www.amcal.com.au/online-catalogue", "Amcal", cata_tracking_writer, last_day_check))

In [9]:
# -------- Salefinder --------

# BCF
get_cata_from_salefinder(driver, "https://salefinder.com.au/BCF-catalogue", "BCF", cata_tracking_writer, last_day_check))

# BigW
get_cata_from_salefinder(driver, "https://salefinder.com.au/Big-W-catalogue", "BigW", cata_tracking_writer, last_day_check))

# Bunnings
get_cata_from_salefinder(driver, "https://salefinder.com.au/Bunnings-Warehouse-catalogue", "Bunnings", cata_tracking_writer, last_day_check))

# IGA
get_cata_from_salefinder(driver, "https://salefinder.com.au/IGA-catalogue", "IGA", cata_tracking_writer, last_day_check))

# IGA Liquor
get_cata_from_salefinder(driver, "https://salefinder.com.au/IGA-Liquor-catalogue", "IGA Liquor", cata_tracking_writer, last_day_check))

# Kmart
get_cata_from_salefinder(driver, "https://salefinder.com.au/Kmart-catalogue", "Kmart", cata_tracking_writer, last_day_check))

# Myer
get_cata_from_salefinder(driver, "https://salefinder.com.au/Myer-catalogue", "Myer", cata_tracking_writer, last_day_check))

# Officeworks
get_cata_from_salefinder(driver, "https://salefinder.com.au/Officeworks-catalogue", "Officeworks", cata_tracking_writer, last_day_check))

# Priceline
get_cata_from_salefinder(driver, "https://salefinder.com.au/Priceline-catalogue", "Priceline", cata_tracking_writer, last_day_check))

# Spotlight
get_cata_from_salefinder(driver, "https://salefinder.com.au/Spotlight-Australia-catalogue", "Spotlight", cata_tracking_writer, last_day_check))

# Supercheap Auto
get_cata_from_salefinder(driver, "https://salefinder.com.au/Supercheap-Auto-catalogue", "Supercheap Auto", cata_tracking_writer, last_day_check))

# Woolworths
get_cata_from_salefinder(driver, "https://salefinder.com.au/Woolworths-catalogue", "Woolworths", cata_tracking_writer, last_day_check))

BCF :  1 catalogue(s)
BigW :  1 catalogue(s)
Bunnings :  No new catalogue available
IGA :  6 catalogue(s)
IGA Liquor :  1 catalogue(s)
Kmart :  1 catalogue(s)
Myer :  No new catalogue available
Officeworks :  1 catalogue(s)
Priceline :  No new catalogue available
Spotlight :  2 catalogue(s)
Supercheap Auto :  1 catalogue(s)
Woolworths :  3 catalogue(s)


In [ ]:
cata_tracking_writer.close()